In [122]:
# Clean Training Pipeline for Tune Embedding Experiments
# This notebook template can be copied for each experiment

import os
import json
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from pathlib import Path
from tensorflow.keras import layers, models
import collections
import random

In [123]:
# ============================================================================
# EXPERIMENT CONFIGURATION
# ============================================================================

# Experiment metadata
EXPERIMENT_NAME = "experiment_0"  # Change this for each experiment
EXPERIMENT_DESCRIPTION = "Baseline model with triplet loss"

In [126]:
# Hyperparameters
CONFIG = {
    "experiment_name": EXPERIMENT_NAME,
    "description": EXPERIMENT_DESCRIPTION,
    "model_params": {
        "emb_dim": 32,
        "rnn_units": 32,
        "dropout_rate": 0.1,  # Add regularization
        "l2_reg": 0.001       # L2 regularization
    },
    "training_params": {
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 20,
        "margin": 0.3,
        "batch_tunes": 16,
        "per_tune": 2
    },
    "data_params": {
        "min_tune_settings": 2,
        "max_sequence_length": 512
    }
}

In [127]:
# ============================================================================
# DIRECTORY SETUP
# ============================================================================

def setup_experiment_directories(experiment_name):
    """Create directory structure for experiment"""
    
    base_dir = Path("..")
    experiment_dir = base_dir / experiment_name
    
    # Create directories
    dirs_to_create = [
        experiment_dir,
        experiment_dir / "checkpoints",
        experiment_dir / "logs",
        experiment_dir / "plots",
        base_dir / "saved_models",
        base_dir / "tokenized_data"
    ]
    
    for dir_path in dirs_to_create:
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created directory: {dir_path}")
    
    return experiment_dir
experiment_dir = setup_experiment_directories(EXPERIMENT_NAME)

✓ Created directory: ../experiment_0
✓ Created directory: ../experiment_0/checkpoints
✓ Created directory: ../experiment_0/logs
✓ Created directory: ../experiment_0/plots
✓ Created directory: ../saved_models
✓ Created directory: ../tokenized_data


In [128]:
# Save experiment configuration
config_path = experiment_dir / "config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)
print(f"✓ Saved config to: {config_path}")

✓ Saved config to: ../experiment_0/config.json


In [129]:
# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================

def create_tune_embedding_model(vocab_size, config):
    """
    Create tune embedding model with proper saving/loading support
    """
    model_params = config["model_params"]
    
    # Define inputs
    notes_in = layers.Input(shape=(None,), dtype="int32", name="note_ids")
    durs_in = layers.Input(shape=(None,), dtype="float32", name="durations")

    # Note embeddings with regularization
    note_emb = layers.Embedding(
        input_dim=vocab_size,
        output_dim=model_params["emb_dim"],
        mask_zero=True,
        embeddings_regularizer=tf.keras.regularizers.l2(model_params["l2_reg"]),
        name="note_embedding"
    )(notes_in)

    # Duration features - using Reshape instead of Lambda for better saving
    dur_feat = layers.Reshape((-1, 1), name="duration_reshape")(durs_in)
    
    # Duration embedding with regularization
    dur_emb = layers.TimeDistributed(
        layers.Dense(
            model_params["emb_dim"], 
            kernel_regularizer=tf.keras.regularizers.l2(model_params["l2_reg"]),
            name="duration_dense"
        ),
        name="duration_embedding"
    )(dur_feat)

    # Combine embeddings
    x = layers.Add(name="combine_embeddings")([note_emb, dur_emb])
    
    # Add dropout for regularization
    x = layers.Dropout(model_params["dropout_rate"], name="embedding_dropout")(x)

    # Bidirectional GRU with regularization
    rnn_out = layers.Bidirectional(
        layers.GRU(
            model_params["rnn_units"], 
            return_sequences=True,
            dropout=model_params["dropout_rate"],
            recurrent_dropout=model_params["dropout_rate"],
            kernel_regularizer=tf.keras.regularizers.l2(model_params["l2_reg"]),
            name="gru"
        ),
        name="bidirectional_gru"
    )(x)

    # Global average pooling
    tune_vec = layers.GlobalAveragePooling1D(name="global_pooling")(rnn_out)
    
    # Additional dense layer before normalization (optional)
    tune_vec = layers.Dense(
        2 * model_params["rnn_units"],
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(model_params["l2_reg"]),
        name="dense_before_norm"
    )(tune_vec)
    
    tune_vec = layers.Dropout(model_params["dropout_rate"], name="final_dropout")(tune_vec)

    # L2 normalization using custom layer instead of Lambda
    class L2Normalize(layers.Layer):
        def __init__(self, **kwargs):
            super(L2Normalize, self).__init__(**kwargs)
        
        def call(self, inputs):
            return tf.math.l2_normalize(inputs, axis=1)
        
        def get_config(self):
            return super(L2Normalize, self).get_config()

    tune_emb = L2Normalize(name="l2_normalize")(tune_vec)

    # Build model
    model = tf.keras.Model(
        inputs=[notes_in, durs_in], 
        outputs=tune_emb, 
        name=f"tune_embedder_{EXPERIMENT_NAME}"
    )
    
    return model

In [130]:
# ============================================================================
# LOSS FUNCTION
# ============================================================================

def batch_hard_triplet_loss(margin=0.3):
    """Batch hard triplet loss"""
    def loss_fn(y_true, y_pred):
        labels = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
        embeddings = y_pred
        
        # Pairwise distances
        dot = tf.matmul(embeddings, embeddings, transpose_b=True)
        sq = tf.reduce_sum(tf.square(embeddings), axis=1, keepdims=True)
        pdist = tf.maximum(sq - 2.0 * dot + tf.transpose(sq), 0.0)
        
        # Masks
        labels_eq = tf.equal(tf.expand_dims(labels,1), tf.expand_dims(labels,0))
        mask_pos = tf.cast(labels_eq, tf.float32) - tf.eye(tf.shape(labels)[0])
        mask_neg = 1.0 - tf.cast(labels_eq, tf.float32)

        # Hard examples
        hardest_pos = tf.reduce_max(pdist * mask_pos, axis=1)
        max_dist = tf.reduce_max(pdist)
        pdist_neg = pdist + max_dist * (1.0 - mask_neg)
        hardest_neg = tf.reduce_min(pdist_neg, axis=1)

        # Triplet loss
        tl = tf.maximum(hardest_pos - hardest_neg + margin, 0.0)
        return tf.reduce_mean(tl)
    
    return loss_fn

In [131]:
# ============================================================================
# DATA LOADING
# ============================================================================

def load_training_data():
    """Load training and validation data for experiments"""
    
    print("📁 Loading training data...")
    
    # Load splits
    train_df = pd.read_pickle('../tokenized_data/train_dataset.pkl')
    val_df = pd.read_pickle('../tokenized_data/val_dataset.pkl')
    
    # Load vocabulary
    with open("../tokenized_data/note_vocab.pkl", "rb") as f:
        vocab_list = pickle.load(f)
    
    # Load split metadata for reference
    with open("../tokenized_data/split_metadata.json", "r") as f:
        split_info = json.load(f)
    
    print(f"✓ Train: {len(train_df):,} samples from {train_df.tune_id.nunique():,} tunes")
    print(f"✓ Val:   {len(val_df):,} samples from {val_df.tune_id.nunique():,} tunes")
    print(f"✓ Vocab: {len(vocab_list)} unique notes")
    
    return train_df, val_df, vocab_list, split_info

def load_test_data():
    """Load test data (only for final evaluation!)"""
    
    print("🔒 Loading test data for final evaluation...")
    
    test_df = pd.read_pickle('../tokenized_data/test_dataset.pkl')
    
    with open("../tokenized_data/note_vocab.pkl", "rb") as f:
        vocab_list = pickle.load(f)
    
    print(f"✓ Test: {len(test_df):,} samples from {test_df.tune_id.nunique():,} tunes")
    print("⚠️  Remember: Use test set only for final model comparison!")
    
    return test_df, vocab_list

In [132]:
# ============================================================================
# UPDATED TRAINING PIPELINE
# ============================================================================

def create_training_dataset_with_validation(train_df, val_df, config):
    """Create both training and validation datasets"""
    
    training_params = config["training_params"]
    
    # Prepare training data (same as before but using train_df)
    by_id = collections.defaultdict(list)

    for notes, durs, tid in zip(train_df.note_ids, train_df.dur_seq, train_df.tune_id):
        by_id[int(tid)].append((notes, durs))
    
    tune_ids = list(by_id.keys())
    
    def balanced_sample_generator():
        while True:
            chosen = random.sample(tune_ids, training_params["batch_tunes"])
            for tid in chosen:
                examples = random.choices(by_id[tid], k=training_params["per_tune"])
                for notes, durs in examples:
                    yield (notes, durs), tid
    
    # Training dataset
    train_ds = tf.data.Dataset.from_generator(
        balanced_sample_generator,
        output_signature=(
            (tf.TensorSpec(shape=(None,), dtype=tf.int32),
             tf.TensorSpec(shape=(None,), dtype=tf.float32)),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    ).padded_batch(
        batch_size=training_params["batch_size"],
        padded_shapes=(([None], [None]), []),
        padding_values=((0, 0.0), 0)
    ).prefetch(tf.data.AUTOTUNE)
    
    # Validation dataset (if val_df has multi-setting tunes)
    val_ds = None
    val_multi_setting = val_df.tune_id.value_counts()
    val_multi_setting_tunes = val_multi_setting[val_multi_setting >= 2].index
    
    if len(val_multi_setting_tunes) > 0:
        val_df_filtered = val_df[val_df.tune_id.isin(val_multi_setting_tunes)]
        
        val_by_id = collections.defaultdict(list)
        for notes, durs, tid in zip(val_df_filtered.note_ids, val_df_filtered.dur_seq, val_df_filtered.tune_id):
            val_by_id[int(tid)].append((notes, durs))
        
        val_tune_ids = list(val_by_id.keys())
        
        def val_generator():
            # Generate a fixed set for validation (not infinite)
            samples = []
            for tid in val_tune_ids:
                examples = random.choices(val_by_id[tid], k=2)  # 2 examples per tune
                for notes, durs in examples:
                    samples.append(((notes, durs), tid))
            
            # Shuffle once
            random.shuffle(samples)
            for sample in samples:
                yield sample
        
        val_ds = tf.data.Dataset.from_generator(
            val_generator,
            output_signature=(
                (tf.TensorSpec(shape=(None,), dtype=tf.int32),
                 tf.TensorSpec(shape=(None,), dtype=tf.float32)),
                tf.TensorSpec(shape=(), dtype=tf.int32)
            )
        ).padded_batch(
            batch_size=training_params["batch_size"],
            padded_shapes=(([None], [None]), []),
            padding_values=((0, 0.0), 0)
        ).prefetch(tf.data.AUTOTUNE)
        
        print(f"✓ Validation dataset created with {len(val_tune_ids)} tunes")
    else:
        print("⚠️  No multi-setting tunes in validation set - skipping validation dataset")
    
    return train_ds, val_ds

In [ ]:

# ============================================================================
# CALLBACKS AND MONITORING
# ============================================================================

def create_callbacks_with_validation(experiment_dir, config, use_validation=True):
    """Create callbacks, optionally including validation monitoring"""
    
    callbacks = []
    
    # Model checkpointing
    if use_validation:
        # Monitor validation loss if available
        monitor_metric = 'val_loss'
        mode = 'min'
    else:
        # Fall back to training loss
        monitor_metric = 'loss'  
        mode = 'min'
    
    # Best model checkpoint
    best_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=experiment_dir / "checkpoints" / "best_weights.weights.h5",
        save_weights_only=True,
        save_best_only=True,
        monitor=monitor_metric,
        mode=mode,
        verbose=1
    )
    callbacks.append(best_checkpoint)
    
    # Regular checkpoints
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=experiment_dir / "checkpoints" / "weights_epoch_{epoch:02d}.weights.h5",
        save_weights_only=True,
        save_freq='epoch',
        verbose=0
    )
    callbacks.append(checkpoint_callback)
    
    # CSV logger
    csv_logger = tf.keras.callbacks.CSVLogger(
        experiment_dir / "logs" / "training_log.csv",
        append=True
    )
    callbacks.append(csv_logger)
    
    # Learning rate reduction
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor=monitor_metric,
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
    callbacks.append(lr_reducer)
    
    # Early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor=monitor_metric,
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    callbacks.append(early_stopping)

    similarity_callback = TuneSimilarityCallback(val_df)
    callbacks.append(similarity_callback)
    
    return callbacks

# ============================================================================
# UPDATED EVALUATION FUNCTIONS
# ============================================================================
def create_evaluation_pairs(df, num_positive=500, num_negative=1000):
    """Create evaluation pairs for validation"""
    
    # Filter tunes with multiple settings
    multi_setting_tunes = df.tune_id.value_counts()
    multi_setting_tunes = multi_setting_tunes[multi_setting_tunes >= 2].index
    
    if len(multi_setting_tunes) == 0:
        return [], []
    
    # Filter DataFrame to only include multi-setting tunes
    df_filtered = df[df.tune_id.isin(multi_setting_tunes)]
    
    # Create positive pairs (same tune, different settings)
    pos_pairs = []
    for tune_id in multi_setting_tunes:
        indices = df_filtered[df_filtered.tune_id == tune_id].index.tolist()
        if len(indices) < 2:
            continue
        random.shuffle(indices)
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                pos_pairs.append((indices[i], indices[j]))
                if len(pos_pairs) >= num_positive:
                    break
            if len(pos_pairs) >= num_positive:
                break
        if len(pos_pairs) >= num_positive:
            break
    
    # Create negative pairs (different tunes)
    neg_pairs = []
    all_indices = df_filtered.index.tolist()
    random.shuffle(all_indices)
    
    for i in range(len(all_indices)):
        for j in range(i + 1, len(all_indices)):
            if df_filtered.iloc[all_indices[i]].tune_id != df_filtered.iloc[all_indices[j]].tune_id:
                neg_pairs.append((all_indices[i], all_indices[j]))
                if len(neg_pairs) >= num_negative:
                    break
        if len(neg_pairs) >= num_negative:
            break
    
    return pos_pairs, neg_pairs
def evaluate_on_validation(model, val_df):
    """Quick evaluation on validation set during training"""
    
    if len(val_df) == 0:
        return {}
    
    # Create evaluation pairs
    pos_pairs, neg_pairs = create_evaluation_pairs(
        val_df, num_positive=min(500, len(val_df)//4), num_negative=min(1000, len(val_df)//2)
    )
    
    if len(pos_pairs) == 0:
        return {"note": "No evaluation possible - insufficient multi-setting tunes"}
    
    # Get embeddings for all required indices
    all_indices = list(set([i for pair in pos_pairs + neg_pairs for i in pair]))
    all_embeddings = {}
    
    for idx in all_indices:
        notes = val_df.iloc[idx].note_ids
        durs = val_df.iloc[idx].dur_seq
        embedding = model.predict([
            np.array([notes]), 
            np.array([durs])
        ], verbose=0)[0]
        all_embeddings[idx] = embedding
    
    # Calculate similarities
    pos_sims = [
        np.dot(all_embeddings[i], all_embeddings[j])
        for i, j in pos_pairs
    ]
    
    neg_sims = [
        np.dot(all_embeddings[i], all_embeddings[j])
        for i, j in neg_pairs
    ]
    
    return {
        "val_positive_similarity": np.mean(pos_sims),
        "val_negative_similarity": np.mean(neg_sims),
        "val_separation": np.mean(pos_sims) - np.mean(neg_sims)
    }

In [ ]:
class TuneSimilarityCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_df, eval_frequency=1):
        super().__init__()
        self.val_df = val_df
        self.eval_frequency = eval_frequency
        self.similarity_history = []
        
    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % self.eval_frequency == 0:
            # Run similarity evaluation
            metrics = evaluate_on_validation(self.model, self.val_df)
            
            # Log metrics
            logs.update({
                'val_pos_similarity': metrics['val_positive_similarity'],
                'val_neg_similarity': metrics['val_negative_similarity'],
                'val_separation': metrics['val_separation']
            })
            
            self.similarity_history.append(metrics)
            
            # Print metrics
            print(f"\nEpoch {epoch+1} Similarity Metrics:")
            print(f"Positive: {metrics['val_positive_similarity']:.3f}")
            print(f"Negative: {metrics['val_negative_similarity']:.3f}")
            print(f"Separation: {metrics['val_separation']:.3f}\n")

In [134]:
# ============================================================================
# SAVING AND LOADING FUNCTIONS
# ============================================================================

def save_model_and_weights(model, experiment_dir, experiment_name):
    """Save model in multiple formats for reliability"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save weights to experiment directory
    weights_path = experiment_dir / "checkpoints" / f"final_weights_{timestamp}.weights.h5"
    model.save_weights(weights_path)
    print(f"✓ Saved weights to: {weights_path}")
    
    # Save weights to main saved_models directory
    main_weights_path = Path("../saved_models") / f"{experiment_name}.weights.h5"
    model.save_weights(main_weights_path)
    print(f"✓ Saved weights to: {main_weights_path}")
    
    # Save model architecture
    architecture_path = experiment_dir / "model_architecture.json"
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"✓ Saved architecture to: {architecture_path}")
    
    # Try to save full model (might fail with Lambda layers)
    try:
        model_path = experiment_dir / "checkpoints" / f"full_model_{timestamp}"
        model.save(model_path, save_format='tf')
        print(f"✓ Saved full model to: {model_path}")
    except Exception as e:
        print(f"⚠ Could not save full model: {e}")
    
    return weights_path, main_weights_path

In [135]:
def load_model_from_experiment(experiment_name, vocab_size=None, checkpoint="best"):
    """Load model from experiment directory"""
    
    experiment_dir = Path(experiment_name)
    
    # Load config
    with open(experiment_dir / "config.json", 'r') as f:
        config = json.load(f)
    
    # Recreate model architecture
    if vocab_size is None:
        # Try to infer vocab size from config or data
        try:
            with open("tokenized_data/note_vocab.pkl", "rb") as f:
                vocab_list = pickle.load(f)
            vocab_size = len(vocab_list) + 2
        except:
            raise ValueError("Could not determine vocab_size. Please provide it explicitly.")
    
    model = create_tune_embedding_model(vocab_size, config)
    
    # Load weights
    if checkpoint == "best":
        weights_path = experiment_dir / "checkpoints" / "best_weights.h5"
    else:
        weights_path = experiment_dir / "checkpoints" / f"final_weights_{checkpoint}.h5"
    
    if weights_path.exists():
        model.load_weights(weights_path)
        print(f"✓ Loaded weights from: {weights_path}")
    else:
        print(f"⚠ Weights file not found: {weights_path}")
        return None
    
    return model, config


In [136]:
# ============================================================================
# MAIN TRAINING FUNCTION
# ============================================================================



print(f"Starting {EXPERIMENT_NAME}")
print("="*50)

# Load data
train_df, val_df, vocab_list, split_info = load_training_data()
print("✓ Data loaded")
vocab_size = len(vocab_list) + 2
# transform into datasets
train_ds, val_ds = create_training_dataset_with_validation(
    train_df, val_df, CONFIG
)
# Create model
model = create_tune_embedding_model(vocab_size, CONFIG)
print("✓ Model created")

# Compile model
optimizer = tf.keras.optimizers.Adam(
    learning_rate=CONFIG["training_params"]["learning_rate"]
)

model.compile(
    optimizer=optimizer,
    loss=batch_hard_triplet_loss(margin=CONFIG["training_params"]["margin"])
)

print("✓ Model compiled")
model.summary()

# Create dataset
train_ds, val_ds = create_training_dataset_with_validation(
    train_df, val_df, CONFIG  # Using the same df for both train and val for simplicity
)
print("✓ Training dataset created")

# Create callbacks
callbacks = create_callbacks_with_validation(
    experiment_dir, CONFIG, use_validation=(val_ds is not None)
)
print("✓ Callbacks created")
steps_per_epoch = len(train_df) // CONFIG["training_params"]["batch_size"]
print(f"Steps per epoch: {steps_per_epoch}")



Starting experiment_0
📁 Loading training data...
✓ Train: 35,415 samples from 15,418 tunes
✓ Val:   7,509 samples from 3,304 tunes
✓ Vocab: 64 unique notes
✓ Data loaded
✓ Validation dataset created with 1396 tunes
✓ Model created
✓ Model compiled


Model: "tune_embedder_experiment_0"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ durations           │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ note_ids            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration_reshape    │ (None, None, 1)   │          0 │ durations[0][0]   │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ note_embedding      │ (None, None, 32)  │      2,112 │ note_ids[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration_embedding  │ (None, None, 32)  │         64 │ duration_reshape… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combine_embeddings  │ (None, None, 32)  │          0 │ note_embedding[0… │
│ (Add)               │                   │            │ duration_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_dropout   │ (None, None, 32)  │          0 │ combine_embeddin… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_gru   │ (None, None, 64)  │     12,672 │ embedding_dropou… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_pooling      │ (None, 64)        │          0 │ bidirectional_gr… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_before_norm   │ (None, 64)        │      4,160 │ global_pooling[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_dropout       │ (None, 64)        │          0 │ dense_before_nor… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalize        │ (None, 64)        │          0 │ final_dropout[0]… │
│ (L2Normalize)       │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,008 (74.25 KB)

 Trainable params: 19,008 (74.25 KB)

 Non-trainable params: 0 (0.00 B)

✓ Validation dataset created with 1396 tunes
✓ Training dataset created
✓ Callbacks created
Steps per epoch: 553


In [ ]:
# Train model
print("Starting training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    epochs=CONFIG["training_params"]["epochs"],
    callbacks=callbacks,
    verbose=1
)


Starting training...
Epoch 1/20
553/553 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.5068
Epoch 1: val_loss improved from inf to 0.31430, saving model to ../experiment_0/checkpoints/best_weights.weights.h5
553/553 ━━━━━━━━━━━━━━━━━━━━ 1318s 2s/step - loss: 0.5067 - val_loss: 0.3143 - learning_rate: 0.0010
Epoch 2/20


2025-06-03 18:34:07.994224: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/tune_embedder_experiment_0_1/duration_embedding_1/loop_body/MatMul/pfor/split/_42]]
2025-06-03 18:34:07.994335: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 6591663739367850741
2025-06-03 18:34:07.994340: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1261855952135355151
2025-06-03 18:34:07.994343: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4658643603350602711
2025-06-03 18:34:07.994345: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7543694434847464891
2025-06-03 18:34:07.994347: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item 

289/553 ━━━━━━━━━━━━━━━━━━━━ 9:17 2s/step - loss: 0.4015

In [ ]:
# Save final model
save_model_and_weights(model, experiment_dir, EXPERIMENT_NAME)

In [ ]:
# Save training history
history_path = experiment_dir / "training_history.pkl"
with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)
print(f"✓ Saved training history to: {history_path}")

print(f"✓ {EXPERIMENT_NAME} completed!")

In [ ]:
import matplotlib.pyplot as plt

# history is the History object returned by model.fit(...)
train_loss = history.history['loss']
val_loss   = history.history.get('val_loss', None)
epochs     = range(1, len(train_loss) + 1)

plt.figure(figsize=(8,4))
plt.plot(epochs, train_loss,  label='Train Loss')
if val_loss is not None:
    plt.plot(epochs, val_loss,  label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True)
plt.show()